## I have used sample data generated by gpt4o. Agent performs better on structured table datasets

## Basic code structure

In [ ]:
# Please intall all the dependencies

In [1]:
import os
os.environ['OPENAI_API_KEY']='please update with your open ai api '

In [2]:
!pip install gradio llama-index-embeddings-openai llama-index-agent-openai llama-index-vector-stores-deeplake llama-index llama-index-core llama-index-multi-modal-llms-openai llama-index-llms-openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.0/326.0 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 15.7 

In [3]:
!pip install deeplake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.4/599.4 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.

In [4]:
import deeplake
from pydantic import BaseModel
from typing import List
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import ServiceContext,VectorStoreIndex
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document
from llama_index.vector_stores.deeplake import DeepLakeVectorStore
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index.core.program import MultiModalLLMCompletionProgram
from llama_index.core.output_parsers import PydanticOutputParser
from llama_index.core.tools import FunctionTool
from llama_index.agent.openai import OpenAIAgent
from dotenv import load_dotenv
from llama_index.core import Settings

In [5]:
# Function tools for the project
def add_guest(guest_info, contact_info):
    # Simulated function to add a guest to the system
    guest_id = len(guest_info) + 1
    guest_data = {
        "guest_id": guest_id,
        "guest_name": guest_info,
        "contact_info": contact_info
    }
    return guest_data

def add_event(event_details):
    # Simulated function to add an event
    event_id = len(event_details) + 1
    return {
        "event_id": event_id,
        **event_details
    }

def register_vendor(vendor_data, contact_info):
    # Simulated function to register a vendor
    vendor_id = len(vendor_data) + 1
    return {
        "vendor_id": vendor_id,
        **vendor_data,
        **contact_info
    }

def register_venue(venue_data, contact_info):
    # Simulated function to register a venue
    venue_id = len(venue_data) + 1
    return {
        "venue_id": venue_id,
        **venue_data,
        **contact_info
    }

def manage_guest_list(event_id, guest_list):
    # Simulated function to manage the guest list
    return {"event_id": event_id, "guest_list": guest_list}

def retrieve_guest_details(identifier, search_by):
    # Simulated function to retrieve guest details
    if search_by == "name":
        if identifier == "Alice":
            return {"guest_id": "GST101", "guest_name": "Alice Johnson"}
        elif identifier == "Bob":
            return {"guest_id": "GST102", "guest_name": "Bob Smith"}
    elif search_by == "id":
        if identifier == "GST101":
            return {"guest_id": "GST101", "guest_name": "Alice Johnson"}
        elif identifier == "GST102":
            return {"guest_id": "GST102", "guest_name": "Bob Smith"}
    return None


def view_vendor_details(identifier, search_by):
    # Simulated function to view vendor details
    if search_by == "name":
        if identifier == "CaterFun":
            return {
                "vendor_id": "VND101",
                "vendor_name": "CaterFun",
                "services_offered": "Kid-friendly catering, birthday party specialists",
                "contact_info": {"Phone": "555-222-1111", "Email": "caterfun@example.com"},
            }
        elif identifier == "EventLightening":
            return {
                "vendor_id": "VND102",
                "vendor_name": "EventLightening",
                "services_offered": "Event lighting and sound services",
                "contact_info": {"Phone": "555-222-1112", "Email": "eventlight@example.com"},
            }
    elif search_by == "id":
        if identifier == "VND101":
            return {
                "vendor_id": "VND101",
                "vendor_name": "CaterFun",
                # ... [Rest of the details]
            }
        elif identifier == "VND102":
            return {
                "vendor_id": "VND102",
                "vendor_name": "EventLightening",
                # ... [Rest of the details]
            }
    return None

def view_venue_details(venue_id):
    # Simulated API call to retrieve venue details
    if venue_id == "VNU101":
        return {
            "venue_name": "Skyline Ballroom",
            "venue_description": "Stunning ballroom with city views, seating up to 300.",
            "venue_location": "North Chicago",
            "contact_info": {"Phone": "555-111-1111", "Email": "skyballroom@example.com"},
            "available_dates": ["2024-08-01", "2024-08-16", "2024-09-01"],
        }
    else:
        return None


def list_all_vendors(service_types=None):
    # Simulated function to list all vendors
    vendors = [
        {"vendor_id": "VND101", "vendor_name": "CaterFun"},
        {"vendor_id": "VND102", "vendor_name": "EventLightening"},
        {"vendor_id": "VND103", "vendor_name": "DecorMasters"},
    ]

    if service_types:
        filtered_vendors = []
        for vendor in vendors:
            for service in service_types:
                if service in vendor["vendor_name"]:
                    filtered_vendors.append(vendor)
        return filtered_vendors
    return vendors


def list_all_events(start_date=None, end_date=None):
    # Simulated function to list all events
    events = [
        {"event_id": "EVT101", "event_name": "Corporate Retreat", "event_date": "2024-08-08"},
        {"event_id": "EVT102", "event_name": "Wedding Gala", "event_date": "2024-08-14"},
    ]

    if start_date:
        return [event for event in events if event["event_date"] >= start_date]
    elif end_date:
        return [event for event in events if event["event_date"] <= end_date]
    return events


def list_all_guests(event_id=None):
    # Simulated function to list all guests
    guests = [
        {"guest_id": "GST101", "guest_name": "Alice Johnson"},
        {"guest_id": "GST102", "guest_name": "Bob Smith"},
        {"guest_id": "GST103", "guest_name": "Carol Davis"},
    ]

    if event_id:
        guest_list = []
        for guest in guests:
            if guest["event_id"] == event_id:
                guest_list.append(guest)
        return guest_list
    return guests


def view_event_details(event_id):
    # Simulated function to view event details
    events_data = {
        "EVT101": {
            "event_id": "EVT101",
            "event_name": "Corporate Retreat",
            "event_location": "Skyline Ballroom",
            "event_date": "2024-08-08",
            "guest_ids": ["GST101", "GST102"],
        },
        "EVT102": {
            "event_id": "EVT102",
            "event_name": "Wedding Reception",
            "event_location": "Garden Paradise",
            "event_date": "2024-08-14",
            "guest_ids": ["GST103", "GST104"],
        },
    }
    return events_data.get(event_id)

def list_all_venues(location=None, available_date=None):
    # Simulated API call to list all venues
    venues = [
        {
            "venue_id": "VNU101",
            "venue_name": "Skyline Ballroom",
            "venue_description": "Ballroom with amazing views",
            "venue_location": "Chicago",
        },
        {
            "venue_id": "VNU102",
            "venue_name": "Garden Paradise",
            "venue_description": "Secluded garden venue, perfect for outdoor events",
            "venue_location": "Suburban Chicago",
        },
    ]

    # Filter venues if location or date is provided
    if location:
        venues = [venue for venue in venues if venue["venue_location"] == location]
    if available_date:
        venues = [venue for venue in venues if available_date in venue["available_dates"]]

    return venues

In [6]:
function_tools = [
    FunctionTool.from_defaults(
        name="add_guest",
        fn=add_guest,
        description="Use this tool to add a guest to the system. Provide guest information and contact information.",
    ),
    FunctionTool.from_defaults(
        name="add_event",
        fn=add_event,
        description="Use this tool to add a new event. Provide event details.",
    ),
    FunctionTool.from_defaults(
        name="register_vendor",
        fn=register_vendor,
        description="Use this tool to register a new vendor. Provide vendor data and contact information.",
    ),
    FunctionTool.from_defaults(
        name="register_venue",
        fn=register_venue,
        description="Use this tool to register a new venue. Provide venue data and contact information.",
    ),
    FunctionTool.from_defaults(
        name="manage_guest_list",
        fn=manage_guest_list,
        description="Use this tool to manage the guest list for an event. Provide event ID and the list of guests.",
    ),
    FunctionTool.from_defaults(
        name="retrieve_guest_details",
        fn=retrieve_guest_details,
        description="Use this tool to retrieve guest details. Provide either guest name or guest ID.",
    ),
    FunctionTool.from_defaults(
        name="view_vendor_details",
        fn=view_vendor_details,
        description="Use this tool to view vendor details. Provide either vendor name or vendor ID.",
    ),
    FunctionTool.from_defaults(
        name="list_all_venues",
        fn=list_all_venues,
        description="Use this tool to list all venues. You can optionally filter by location or available date.",
    ),
    FunctionTool.from_defaults(
        name="list_all_vendors",
        fn=list_all_vendors,
        description="Use this tool to list all vendors. You can optionally filter by service types.",
    ),
    FunctionTool.from_defaults(
        name="list_all_events",
        fn=list_all_events,
        description="Use this tool to list all events. You can optionally filter by start date or end date.",
    ),
    FunctionTool.from_defaults(
        name="list_all_guests",
        fn=list_all_guests,
        description="Use this tool to list all guests. You can optionally provide an event ID to list guests for a specific event.",
    ),
    FunctionTool.from_defaults(
        name="view_event_details",
        fn=view_event_details,
        description="Use this tool to view details of an event. Provide the event ID.",
    ),
    FunctionTool.from_defaults(
        name="view_venue_details",
        fn=view_venue_details,
        description="Use this tool to view details of a venue. Provide the venue ID.",
    )
]

In [ ]:
# tools = [
#     add_guest,
#     add_event,
#     register_vendor,
#     register_venue,
#     manage_guest_list,
#     retrieve_guest_details,
#     view_vendor_details,
#     list_all_venues,
#     list_all_vendors,
#     list_all_events,
#     list_all_guests,
#     view_event_details,
#     view_venue_details
# ]

In [7]:
# Initialize the language model with specific parameters suited for the task
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.2)

# Create an instance of the OpenAIAgent with appropriate tools and system prompt
agent = OpenAIAgent.from_tools(
    system_prompt="""
    You are an efficient Event Scheduler employed by a prestigious event management company. Your primary responsibility is to use your expertise in organizing events to ensure smooth coordination and timely execution, exceeding client expectations.

    When a user approaches you with an event-related query, your first step is to understand their event requirements, including the type of event, preferred dates, venue preferences, and estimated number of guests. Analyze the context and intent behind their query to provide tailored recommendations and support throughout the event planning process.

    Your role is to be a reliable and efficient event planner that users can depend on to organize memorable and successful events. Use the available tools to query the database for venues, vendors, and guest details, and ensure every detail is meticulously planned.

    Include options for venues and vendors based on user preferences and budget, and calculate the estimated total cost for the events you propose.
    """,

    tools=function_tools,
    llm=llm,
    verbose=True,
)


# r = agent.chat("Hi")
# print(str(r))

In [8]:
r = agent.chat("Hi")
print(str(r))

Added user message to memory: Hi
Hello! How can I assist you today with event planning or organization?


In [16]:
r = agent.chat("please find details of venue")
print(str(r))

Added user message to memory: please find details of venue
To find the details of a venue, I will need the venue ID. Could you please provide me with the venue ID so that I can retrieve the venue details for you?


In [17]:
r = agent.chat("please find details for event id EVT101")
print(str(r))

Added user message to memory: please find details for event id EVT101
=== Calling Function ===
Calling function: view_event_details with args: {"event_id":"EVT101"}
Got output: {'event_id': 'EVT101', 'event_name': 'Corporate Retreat', 'event_location': 'Skyline Ballroom', 'event_date': '2024-08-08', 'guest_ids': ['GST101', 'GST102']}

The details for the event with ID EVT101 are as follows:
- Event Name: Corporate Retreat
- Event Location: Skyline Ballroom
- Event Date: August 8, 2024
- Guest IDs: GST101, GST102

If you need more information or assistance with this event, feel free to let me know.


In [15]:
r = agent.chat(" for guest id GST101")
print(str(r))

Added user message to memory:  for guest id GST101
=== Calling Function ===
Calling function: list_all_guests with args: {"event_id":"GST101"}
Got output: Error: 'event_id'

=== Calling Function ===
Calling function: list_all_guests with args: {"event_id":"GST101"}
Got output: Error: 'event_id'

=== Calling Function ===
Calling function: list_all_guests with args: {"event_id":"GST101"}
Got output: Error: 'event_id'

I apologize for the inconvenience. It seems there is an issue with retrieving the guest list for the event with the ID GST101. If you have any other event IDs or if there is anything else I can assist you with, please let me know.


In [19]:
r = agent.chat(" please add a guest in list ")
print(str(r))

Added user message to memory:  please add a guest in list 
To add a guest to the guest list, I will need the guest information and contact details. Could you please provide me with the necessary details of the guest you would like to add?


In [20]:
r = agent.chat(" his name vishal meena. guest id GST105 contant number 9988776655")
print(str(r))

Added user message to memory:  his name vishal meena. guest id GST105 contant number 9988776655
=== Calling Function ===
Calling function: add_guest with args: {"guest_info":{"guest_name":"Vishal Meena","guest_id":"GST105"},"contact_info":{"contact_number":"9988776655"}}
Got output: {'guest_id': 3, 'guest_name': {'guest_name': 'Vishal Meena', 'guest_id': 'GST105'}, 'contact_info': {'contact_number': '9988776655'}}

The guest with the name Vishal Meena has been successfully added to the guest list with the guest ID GST105 and contact number 9988776655. If you need any further assistance or have more guests to add, feel free to let me know.


In [21]:
r = agent.chat(" please find the details of gst105")
print(str(r))

Added user message to memory:  please find the details of gst105
=== Calling Function ===
Calling function: retrieve_guest_details with args: {"identifier":"GST105","search_by":"guest_id"}
Got output: None

I couldn't find any details for the guest with the ID GST105. If you have any other guest IDs or names you would like me to search for, please let me know.


In [18]:
r = agent.chat(" please find vendor details for VND101")
print(str(r))

Added user message to memory:  please find vendor details for VND101
=== Calling Function ===
Calling function: view_vendor_details with args: {"identifier":"VND101","search_by":"vendor_id"}
Got output: None

I couldn't find any details for the vendor with the ID VND101. If you have any other vendor IDs or names you would like me to search for, please let me know.
